<a href="https://colab.research.google.com/github/shouryasharmahere/agents-course/blob/main/notebooks/unit2/smolagents/multiagent_notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Solving a complex task with a multi-agent hierarchy

This notebook is part of the [Hugging Face Agents Course](https://www.hf.co/learn/agents-course), a free Course from beginner to expert, where you learn to build Agents.

![Agents course share](https://huggingface.co/datasets/agents-course/course-images/resolve/main/en/communication/share.png)

The reception is approaching! With your help, Alfred is now nearly finished with the preparations.

But now there's a problem: the Batmobile has disappeared. Alfred needs to find a replacement, and find it quickly.

Fortunately, a few biopics have been done on Bruce Wayne's life, so maybe Alfred could get a car left behind on one of the movie set, and re-engineer it up to modern standards, which certainly would include a full self-driving option.

But this could be anywhere in the filming locations around the world - which could be numerous.

So Alfred wants your help. Could you build an agent able to solve this task?

> 👉 Find all Batman filming locations in the world, calculate the time to transfer via boat to there, and represent them on a map, with a color varying by boat transfer time. Also represent some supercar factories with the same boat transfer time.

Let's build this!

In [7]:
!pip install 'smolagents[litellm]' matplotlib geopandas shapely kaleido -q

In [8]:
from huggingface_hub import notebook_login

notebook_login()

In [9]:
# We first make a tool to get the cargo plane transfer time.
import math
from typing import Optional, Tuple

from smolagents import tool


@tool
def calculate_cargo_travel_time(
    origin_coords: Tuple[float, float],
    destination_coords: Tuple[float, float],
    cruising_speed_kmh: Optional[float] = 750.0,  # Average speed for cargo planes
) -> float:
    """
    Calculate the travel time for a cargo plane between two points on Earth using great-circle distance.

    Args:
        origin_coords: Tuple of (latitude, longitude) for the starting point
        destination_coords: Tuple of (latitude, longitude) for the destination
        cruising_speed_kmh: Optional cruising speed in km/h (defaults to 750 km/h for typical cargo planes)

    Returns:
        float: The estimated travel time in hours

    Example:
        >>> # Chicago (41.8781° N, 87.6298° W) to Sydney (33.8688° S, 151.2093° E)
        >>> result = calculate_cargo_travel_time((41.8781, -87.6298), (-33.8688, 151.2093))
    """

    def to_radians(degrees: float) -> float:
        return degrees * (math.pi / 180)

    # Extract coordinates
    lat1, lon1 = map(to_radians, origin_coords)
    lat2, lon2 = map(to_radians, destination_coords)

    # Earth's radius in kilometers
    EARTH_RADIUS_KM = 6371.0

    # Calculate great-circle distance using the haversine formula
    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = (
        math.sin(dlat / 2) ** 2
        + math.cos(lat1) * math.cos(lat2) * math.sin(dlon / 2) ** 2
    )
    c = 2 * math.asin(math.sqrt(a))
    distance = EARTH_RADIUS_KM * c

    # Add 10% to account for non-direct routes and air traffic controls
    actual_distance = distance * 1.1

    # Calculate flight time
    # Add 1 hour for takeoff and landing procedures
    flight_time = (actual_distance / cruising_speed_kmh) + 1.0

    # Format the results
    return round(flight_time, 2)


print(calculate_cargo_travel_time((41.8781, -87.6298), (-33.8688, 151.2093)))

22.82


For the model provider, we use Together AI, one of the new [inference providers on the Hub](https://huggingface.co/blog/inference-providers)!

Regarding the GoogleSearchTool: this requires either having setup env variable `SERPAPI_API_KEY` and passing `provider="serpapi"` or having `SERPER_API_KEY` and passing `provider=serper`.

If you don't have any Serp API provider setup, you can use `DuckDuckGoSearchTool` but beware that it has a rate limit.

In [10]:
import os
from PIL import Image
from smolagents import CodeAgent, GoogleSearchTool, HfApiModel, VisitWebpageTool


model = HfApiModel(model_id="Qwen/Qwen2.5-Coder-32B-Instruct", provider="together")

We can start with creating a baseline, simple agent to give us a simple report.

In [11]:
task = """Find all Batman filming locations in the world, calculate the time to transfer via cargo plane to here (we're in Gotham, 40.7128° N, 74.0060° W), and return them to me as a pandas dataframe.
Also give me some supercar factories with the same cargo plane transfer time."""

In [12]:
from google.colab import userdata
import os
os.environ["SERPAPI_API_KEY"] = userdata.get('SERPAPI_API_KEY')

In [13]:
agent = CodeAgent(
    model=model,
    tools=[GoogleSearchTool(), VisitWebpageTool(), calculate_cargo_travel_time],
    additional_authorized_imports=["pandas"],
    max_steps=20,
)

In [14]:
result = agent.run(task)

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ Find all Batman filming locations in the world, calculate the time to transfer via cargo plane to here (we're   │
│ in Gotham, 40.7128° N, 74.0060° W), and return them to me as a pandas dataframe.                                │
│ Also give me some supercar factories with the same cargo plane transfer time.                                   │
│                                                                                                                 │
╰─ HfApiModel - Qwen/Qwen2.5-Coder-32B-Instruct ──────────────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  batman_locations = web_search(query="Batman filming locations")                                                  
  print(batman_locations)                                                                                          
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Execution logs:
## Search Results
0. [The Batman (2022) - Filming & production](https://www.imdb.com/title/tt1877830/locations/)
Source: IMDb

Filming locations: Necropolis Cemetery, Glasgow, Scotland, UK (Batman and Selina leaving the cemetery) Helpful•86 1
St. George's Hall, Liverpool, England, UK

1. [The Batman | Film Locations](https://movie-locations.com/movies/b/The-Batman-2022-2.php)
Source: The Worldwide Guide To Movie Locations

Film locations for The Batman (2022) in Liverpool, London, Glasgow and Chicago.

2. [Where was The Batman filmed? ALL the Filming Locations 
...](https://www.atlasofwonders.com/2022/04/where-was-the-batman-filmed.html)
Source: Atlas of Wonders

Guide to the all the filming locations of The Batman - The 2022 movie with Robert Pattison filmed in Chicago and 
The UK - The City and the Wayne House.

3. [Batman (1989) - Filming & production](https://www.imdb.com/title/tt0096895/locations/)
Source: IMDb

Filming locations ; Knebworth House, Knebworth, Hertfordshire, England, UK. (Wayne Manor; exterior) ; Acton Lane 
Power Station, Acton Lane, Acton, London, England ...

4. [12 Batman Movie Locations You Can 
Visit!](https://www.travelandleisureasia.com/global/destinations/batman-movie-locations-you-can-visit/)
Date published: Jan 10, 2023
Source: Travel and Leisure Asia

Here are 12 real-world Batman movie locations that you can visit now · St. George's Hall, Liverpool · Two Temple 
Place, London · Wollaton Hall, ...

5. [Dark Knight Rises Tour: See Batman Movie 
Locations](https://www.travelchannel.com/interests/arts-and-culture/photos/see-batman-movie-locations)
Source: Travel Channel

See Batman Movie Locations · Wollaton Hall · Carnegie Mellon University · The Farmiloe Building · Queensboro Bridge
· Wall Street · Mehrangarh Fort · Turda ...

6. [Batman Movies and Filming Locations in Los 
Angeles](https://www.traveltodayla.com/post/batman-movies-in-la?srsltid=AfmBOoruY_HLuIghogndgX6YPgiOcMqZ8kpjah3qetM
fhzH5LKk2whRa)
Source: LA Today

Westwood, Los Angeles, CA. Step into the world of Batman and Robin with a visit to the Kirkeby Center, the location
of a thrilling scene from the 1966 film ...

7. [The Dark Knight | 2008](https://movie-locations.com/movies/d/Dark-Knight.php)
Source: The Worldwide Guide To Movie Locations

Discover where The Dark Knight (2008) was filmed around Chicago, as well as in London and Bedfordshire in the UK, 
and briefly in Hong Kong.

Out: None

...

[Step 5: Duration 22.49 seconds| Input tokens: 25,921 | Output tokens: 4,425]

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 6 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  # Manually specify the coordinates for supercar factories                                                        
  supercar_coords = {                                                                                              
      "Ferrari, Maranello, Italy": (44.5403, 10.9921),                                                             
      "Lamborghini, Sant'Agata Bolognese, Italy": (44.5208, 11.3264),                                              
      "Pagani, San Cesario sul Panaro, Italy": (44.6423, 10.9917),                                                 
      "Ducati, Bologna, Italy": (44.4937, 11.3426),                                                                
      "Bugatti, Molsheim, France": (48.6919, 7.7495),                                                              
      "SSC North America, Richland, Washington, USA": (46.1636, -119.2751),                                        
      "NSX Factory, Marysville, Ohio, USA": (41.3851, -83.5364)                                                    
  }                                                                                                                
                                                                                                                   
  # Calculate travel time for each supercar factory                                                                
  supercar_travel_times = {}                                                                                       
  for loc, coords in supercar_coords.items():                                                                      
      travel_time = calculate_cargo_travel_time(origin_coords=coords, destination_coords=gotham_coords)            
      supercar_travel_times[loc] = travel_time                                                                     
                                                                                                                   
  # Create a DataFrame for supercar factories                                                                      
  supercar_df = pd.DataFrame(list(supercar_travel_times.items()), columns=['Location', 'Travel Time (hours)'])     
                                                                                                                   
  # Filter supercar factories to match the travel times of Batman filming locations                                
  matching_supercar_df = supercar_df[supercar_df['Travel Time (hours)'].isin(batman_df['Travel Time (hours)'])]    
                                                                                                                   
  # Combine Batman filming locations and matching supercar factories into a single DataFrame                       
  final_df = pd.concat([batman_df, matching_supercar_df], ignore_index=True)                                       
                                                                                                                   
  # Print the final DataFrame                                                                                      
  print(final_df)                                                                                                  
                                                                                                                   
  # Return the final DataFrame as the answer                                                                       
  final_answer(final_df)                                                                                           
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Execution logs:
                                             Location  Travel Time (hours)
0          Necropolis Cemetery, Glasgow, Scotland, UK                 8.60
1           St. George's Hall, Liverpool, England, UK                 8.81
2                   Two Temple Place, London, England                 9.17
3                  Wollaton Hall, Nottingham, England                 9.00
4   Knebworth House, Knebworth, Hertfordshire, Eng...                 9.14
5   Acton Lane Power Station, Acton Lane, Acton, L...                 9.16
6   Carnegie Mellon University, Pittsburgh, Pennsy...                 1.74
7              The Farmiloe Building, London, England                 9.17
8               Queensboro Bridge, New York City, USA                 1.01
9                     Wall Street, New York City, USA                 1.00
10         Mehrangarh Fort, Jodhpur, Rajasthan, India                18.34
11                        Turda Gorge, Turda, Romania                11.89
12     Kirkeby Center, Westwood, Los Angeles, CA, USA                 6.79

Out - Final answer:                                              Location  Travel Time (hours)
0          Necropolis Cemetery, Glasgow, Scotland, UK                 8.60
1           St. George's Hall, Liverpool, England, UK                 8.81
2                   Two Temple Place, London, England                 9.17
3                  Wollaton Hall, Nottingham, England                 9.00
4   Knebworth House, Knebworth, Hertfordshire, Eng...                 9.14
5   Acton Lane Power Station, Acton Lane, Acton, L...                 9.16
6   Carnegie Mellon University, Pittsburgh, Pennsy...                 1.74
7              The Farmiloe Building, London, England                 9.17
8               Queensboro Bridge, New York City, USA                 1.01
9                     Wall Street, New York City, USA                 1.00
10         Mehrangarh Fort, Jodhpur, Rajasthan, India                18.34
11                        Turda Gorge, Turda, Romania                11.89
12     Kirkeby Center, Westwood, Los Angeles, CA, USA                 6.79

[Step 6: Duration 13.10 seconds| Input tokens: 37,362 | Output tokens: 5,456]

In [ ]:
result

,Location,Travel Time to Gotham (hours)
0,"Necropolis Cemetery, Glasgow, Scotland, UK",8.60
1,"St. George's Hall, Liverpool, England, UK",8.81
2,"Two Temple Place, London, England, UK",9.17
3,"Wollaton Hall, Nottingham, England, UK",9.00
4,"Knebworth House, Knebworth, Hertfordshire, Eng...",9.15
5,"Acton Lane Power Station, Acton Lane, Acton, E...",9.16
6,"Queensboro Bridge, New York City, USA",1.01
7,"Wall Street, New York City, USA",1.00
8,"Mehrangarh Fort, Jodhpur, Rajasthan, India",18.34
9,"Turda Gorge, Turda, Romania",11.89


We could already improve this a bit by throwing in some dedicated planning steps, and adding more prompting.

In [15]:
agent.planning_interval = 4

detailed_report = agent.run(f"""
You're an expert analyst. You make comprehensive reports after visiting many websites.
Don't hesitate to search for many queries at once in a for loop.
For each data point that you find, visit the source url to confirm numbers.

{task}
""")

print(detailed_report)

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ You're an expert analyst. You make comprehensive reports after visiting many websites.                          │
│ Don't hesitate to search for many queries at once in a for loop.                                                │
│ For each data point that you find, visit the source url to confirm numbers.                                     │
│                                                                                                                 │
│ Find all Batman filming locations in the world, calculate the time to transfer via cargo plane to here (we're   │
│ in Gotham, 40.7128° N, 74.0060° W), and return them to me as a pandas dataframe.                                │
│ Also give me some supercar factories with the same cargo plane transfer time.                                   │
│                                                                                                                 │
╰─ HfApiModel - Qwen/Qwen2.5-Coder-32B-Instruct ──────────────────────────────────────────────────────────────────╯

────────────────────────────────────────────────── Initial plan ───────────────────────────────────────────────────
Here is the plan of action that I will follow to solve the task:
```
1. Perform a web search to find a comprehensive list of Batman filming locations worldwide.
2. For each filming location, visit the source URL to confirm the coordinates.
3. Perform a web search to find a comprehensive list of supercar factories worldwide.
4. For each supercar factory, visit the source URL to confirm the coordinates.
5. Calculate the cargo plane transfer time from Gotham to each Batman filming location.
6. Calculate the cargo plane transfer time from Gotham to each supercar factory.
7. Organize the collected data into a pandas dataframe with columns for location name, coordinates, distance from 
Gotham, and estimated transfer time.
8. Finalize the pandas dataframe to include both Batman filming locations and supercar factories with the same 
cargo plane transfer time.
9. Provide the final answer as the pandas dataframe.


```

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  batman_filming_locations = web_search(query="Batman filming locations worldwide")                                
  print(batman_filming_locations)                                                                                  
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Execution logs:
## Search Results
0. [12 Batman Movie Locations You Can 
Visit!](https://www.travelandleisureasia.com/global/destinations/batman-movie-locations-you-can-visit/)
Date published: Jan 10, 2023
Source: Travel and Leisure Asia

Most of the filming of Batman movies is done in the Warner Bros studios and across the US, including New York and 
Pittsburgh. You will notice ...

1. [Category:Film Locations | Batman Wiki - Fandom](https://batman.fandom.com/wiki/Category:Film_Locations)
Source: Batman Wiki

Film Locations · A. Arctic World · B Batcave (Burtonverse) · C Cave of Horrors · D Category:Dark Knight Trilogy 
locations · F.

2. [The Batman (2022) - Filming & production](https://www.imdb.com/title/tt1877830/locations/)
Source: IMDb

Filming locations: Necropolis Cemetery, Glasgow, Scotland, UK (Batman and Selina leaving the cemetery) Helpful•86 1
St. George's Hall, Liverpool, England, UK

3. [The Batman | Film Locations](https://movie-locations.com/movies/b/The-Batman-2022-2.php)
Source: The Worldwide Guide To Movie Locations

Film locations for The Batman (2022) in Liverpool, London, Glasgow and Chicago.

4. [Dark Knight Rises Tour: See Batman Movie 
Locations](https://www.travelchannel.com/interests/arts-and-culture/photos/see-batman-movie-locations)
Source: Travel Channel

See Batman Movie Locations · Wollaton Hall · Carnegie Mellon University · The Farmiloe Building · Queensboro Bridge
· Wall Street · Mehrangarh Fort · Turda Saline.

5. [What cities in America and anywhere in the world do you 
...](https://www.reddit.com/r/batman/comments/1d1t88q/what_cities_in_america_and_anywhere_in_the_world/)
Source: Reddit · r/batman

Glasgow and Liverpool were used in the shoot for The Batman. I'd avoid Chicago, as that looks more like Metropolis 
than Gotham.

6. [Where was The Batman filmed? ALL the Filming Locations 
...](https://www.atlasofwonders.com/2022/04/where-was-the-batman-filmed.html)
Source: Atlas of Wonders

The Batman was primarily filmed in the United Kingdom. Most of the recognizable buildings of this new version of 
Gotham City are located in Liverpool.

7. [The Dark Knight | 2008](https://movie-locations.com/movies/d/Dark-Knight.php)
Source: The Worldwide Guide To Movie Locations

Discover where The Dark Knight (2008) was filmed around Chicago, as well as in London and Bedfordshire in the UK, 
and briefly in Hong Kong.

8. [The Dark Knight Rises](https://onlocationtours.com/locations/the-dark-knight-rises/)
Source: On Location Tours

The 2012 end of Christopher Nolan's Batman trilogy, The Dark Knight Rises, was filmed in New York City. Filming 
locations can be seen on The Super Tour of NYC.

9. [The Worldwide Guide to Movie Locations - Batman Begins 
...](https://m.facebook.com/MovieLocations/photos/batman-begins-2005-senate-house-university-of-london-bloomsbury-l
ondon-wc1-bruce/940979554684554/)
Source: Facebook · The Worldwide Guide to Movie Locations

The Worldwide Guide to Movie Locations - Batman Begins (2005) Senate House, University of London, Bloomsbury, 
London WC1. Bruce Wayne ...

Out: None

...

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 4 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  import re                                                                                                        
  import pandas as pd                                                                                              
                                                                                                                   
  # Batman filming locations and their coordinates                                                                 
  batman_locations = {                                                                                             
      "Wall Street in Manhattan": None,                                                                            
      "the Trump Tower in Manhattan": None,                                                                        
      "One Liberty Plaza on 165 Broadway in New York": None                                                        
  }                                                                                                                
                                                                                                                   
  # Search for coordinates of Batman filming locations                                                             
  for location in batman_locations:                                                                                
      search_query = f"coordinates of {location}"                                                                  
      coordinates = web_search(query=search_query)                                                                 
      # Extract coordinates using regex                                                                            
      match = re.search(r'(\d+\.\d+°\s[N|S],\s\d+\.\d+°\s[E|W])', coordinates)                                     
      if match:                                                                                                    
          batman_locations[location] = match.group(1)                                                              
                                                                                                                   
  # Supercar factories and their coordinates                                                                       
  supercar_factories = {                                                                                           
      "Lamborghini": None,                                                                                         
      "Ferrari": None,                                                                                             
      "Porsche": None,                                                                                             
      "Pagani": None,                                                                                              
      "Koenigsegg": None,                                                                                          
      "McLaren": None,                                                                                             
      "Aston Martin": None,                                                                                        
      "Bugatti": None,                                                                                             
      "Rimac": None,                                                                                               
      "Enzo Ferrari": None                                                                                         
  }                                                                                                                
                                                                                                                   
  # Search for coordinates of supercar factories         

Execution logs:
Empty DataFrame
Columns: []
Index: []

Out - Final answer: Empty DataFrame
Columns: []
Index: []

[Step 4: Duration 58.03 seconds| Input tokens: 35,595 | Output tokens: 2,585]

Empty DataFrame
Columns: []
Index: []


In [17]:
detailed_report

""


from matplotlib import pyplot as plt
import seaborn as sns
def _plot_series(series, series_name, series_index=0):
  palette = list(sns.palettes.mpl_palette('Dark2'))
  counted = (series['index']
                .value_counts()
              .reset_index(name='counts')
              .rename({'index': 'index'}, axis=1)
              .sort_values('index', ascending=True))
  xs = counted['index']
  ys = counted['counts']
  plt.plot(xs, ys, label=series_name, color=palette[series_index % len(palette)])

fig, ax = plt.subplots(figsize=(10, 5.2), layout='constrained')
df_sorted = _df_0.sort_values('index', ascending=True)
_plot_series(df_sorted, '')
sns.despine(fig=fig, ax=ax)
plt.xlabel('index')
_ = plt.ylabel('count()')

Thanks to these quick changes, we obtained a much more concise report by simply providing our agent a detailed prompt, and giving it planning capabilities!

💸 But as you can see, the context window is quickly filling up. So **if we ask our agent to combine the results of detailed search with another, it will be slower and quickly ramp up tokens and costs**.

➡️ We need to improve the structure of our system.

## ✌️ Splitting the task between two agents

Multi-agent structures allow to separate memories between different sub-tasks, with two great benefits:
- Each agent is more focused on its core task, thus more performant
- Separating memories reduces the count of input tokens at each step, thus reducing latency and cost.

Let's create a team with a dedicated web search agent, managed by another agent.

The manager agent should have plotting capabilities to redact its final report: so let us give it access to additional imports, including `matplotlib`, and `geopandas` + `shapely` for spatial plotting.

In [18]:
model = HfApiModel(
    "Qwen/Qwen2.5-Coder-32B-Instruct", provider="together", max_tokens=8096
)
os.environ["SERPER_API_KEY"] = userdata.get('SERPER_API_KEY')
web_agent = CodeAgent(
    model=model,
    tools=[
        GoogleSearchTool(provider="serper"),
        VisitWebpageTool(),
        calculate_cargo_travel_time,
    ],
    name="web_agent",
    description="Browses the web to find information",
    verbosity_level=0,
    max_steps=10,
)

The manager agent will need to do some mental heavy lifting.

So we give it the stronger model [DeepSeek-R1](https://huggingface.co/deepseek-ai/DeepSeek-R1), and add a `planning_interval` to the mix.

In [19]:
from google.colab import userdata
import os
os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')

In [28]:
from smolagents.utils import encode_image_base64, make_image_url
from smolagents import OpenAIServerModel


def check_reasoning_and_plot(final_answer, agent_memory):
    final_answer
    multimodal_model = OpenAIServerModel("gpt-4o", max_tokens=8096)
    filepath = "saved_map.png"
    assert os.path.exists(filepath), "Make sure to save the plot under saved_map.png!"
    image = Image.open(filepath)
    prompt = (
        f"Here is a user-given task and the agent steps: {agent_memory.get_succinct_steps()}. Now here is the plot that was made."
        "Please check that the reasoning process and plot are correct: do they correctly answer the given task?"
        "First list reasons why yes/no, then write your final decision: PASS in caps lock if it is satisfactory, FAIL if it is not."
        "Don't be harsh: if the plot mostly solves the task, it should pass."
        "To pass, a plot should be made using px.scatter_map and not any other method (scatter_map looks nicer)."
    )
    messages = [
        {
            "role": "user",
            "content": [
                {
                    "type": "text",
                    "text": prompt,
                },
                {
                    "type": "image_url",
                    "image_url": {"url": make_image_url(encode_image_base64(image))},
                },
            ],
        }
    ]
    output = multimodal_model(messages).content
    print("Feedback: ", output)
    if "FAIL" in output:
        raise Exception(output)
    return True


manager_agent = CodeAgent(
    # model=HfApiModel("deepseek-ai/DeepSeek-R1", provider="together", max_tokens=8096),
    model=HfApiModel("deepseek-ai/DeepSeek-R1-Distill-Qwen-32B", max_tokens=8096),
    tools=[calculate_cargo_travel_time],
    managed_agents=[web_agent],
    additional_authorized_imports=[
        "geopandas",
        "plotly",
        "shapely",
        "json",
        "pandas",
        "numpy",
    ],
    planning_interval=5,
    verbosity_level=2,
    final_answer_checks=[check_reasoning_and_plot],
    max_steps=15,
)

Let us inspect what this team looks like:

In [29]:
manager_agent.visualize()

CodeAgent | deepseek-ai/DeepSeek-R1-Distill-Qwen-32B
├── ✅ Authorized imports: ['geopandas', 'plotly', 'shapely', 'json', 'pandas', 'numpy']
├── 🛠️ Tools:
│   ┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
│   ┃ Name                        ┃ Description                           ┃ Arguments                             ┃
│   ┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│   │ calculate_cargo_travel_time │ Calculate the travel time for a cargo │ origin_coords (`array`): Tuple of     │
│   │                             │ plane between two points on Earth     │ (latitude, longitude) for the         │
│   │                             │ using great-circle distance.          │ starting point                        │
│   │                             │                                       │ destination_coords (`array`): Tuple   │
│   │                             │                                       │ of (latitude, longitude) for the      │
│   │                             │                                       │ destination                           │
│   │                             │                                       │ cruising_speed_kmh (`number`):        │
│   │                             │                                       │ Optional cruising speed in km/h       │
│   │                             │                                       │ (defaults to 750 km/h for typical     │
│   │                             │                                       │ cargo planes)                         │
│   │ final_answer                │ Provides a final answer to the given  │ answer (`any`): The final answer to   │
│   │                             │ problem.                              │ the problem                           │
│   └─────────────────────────────┴───────────────────────────────────────┴───────────────────────────────────────┘
└── 🤖 Managed agents:
    └── web_agent | CodeAgent | Qwen/Qwen2.5-Coder-32B-Instruct
        ├── ✅ Authorized imports: []
        ├── 📝 Description: Browses the web to find information
        └── 🛠️ Tools:
            ┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
            ┃ Name                        ┃ Description                       ┃ Arguments                         ┃
            ┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
            │ web_search                  │ Performs a google web search for  │ query (`string`): The search      │
            │                             │ your query then returns a string  │ query to perform.                 │
            │                             │ of the top search results.        │ filter_year (`integer`):          │
            │                             │                                   │ Optionally restrict results to a  │
            │                             │                                   │ certain year                      │
            │ visit_webpage               │ Visits a webpage at the given url │ url (`string`): The url of the    │
            │                             │ and reads its content as a        │ webpage to visit.                 │
            │                             │ markdown string. Use this to      │                                   │
            │                             │ browse webpages.                  │                                   │
            │ calculate_cargo_travel_time │ Calculate the travel time for a   │ origin_coords (`array`): Tuple of │
            │                             │ cargo plane between two points on │ (latitude, longitude) for the     │
            │                             │ Earth using great-circle          │ starting point                    │
            │                            

In [ ]:
manager_agent.run("""
Find all Batman filming locations in the world, calculate the time to transfer via cargo plane to here (we're in Gotham, 40.7128° N, 74.0060° W).
Also give me some supercar factories with the same cargo plane transfer time. You need at least 6 points in total.
Represent this as spatial map of the world, with the locations represented as scatter points with a color that depends on the travel time, and save it to saved_map.png!

Here's an example of how to plot and return a map:
import plotly.express as px
df = px.data.carshare()
fig = px.scatter_map(df, lat="centroid_lat", lon="centroid_lon", text="name", color="peak_hour", size=100,
     color_continuous_scale=px.colors.sequential.Magma, size_max=15, zoom=1)
fig.show()
fig.write_image("saved_image.png")
final_answer(fig)

Never try to process strings using code: when you have a string to read, just print it and you'll see it.
""")

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ Find all Batman filming locations in the world, calculate the time to transfer via cargo plane to here (we're   │
│ in Gotham, 40.7128° N, 74.0060° W).                                                                             │
│ Also give me some supercar factories with the same cargo plane transfer time. You need at least 6 points in     │
│ total.                                                                                                          │
│ Represent this as spatial map of the world, with the locations represented as scatter points with a color that  │
│ depends on the travel time, and save it to saved_map.png!                                                       │
│                                                                                                                 │
│ Here's an example of how to plot and return a map:                                                              │
│ import plotly.express as px                                                                                     │
│ df = px.data.carshare()                                                                                         │
│ fig = px.scatter_map(df, lat="centroid_lat", lon="centroid_lon", text="name", color="peak_hour", size=100,      │
│      color_continuous_scale=px.colors.sequential.Magma, size_max=15, zoom=1)                                    │
│ fig.show()                                                                                                      │
│ fig.write_image("saved_image.png")                                                                              │
│ final_answer(fig)                                                                                               │
│                                                                                                                 │
│ Never try to process strings using code: when you have a string to read, just print it and you'll see it.       │
│                                                                                                                 │
╰─ HfApiModel - deepseek-ai/DeepSeek-R1-Distill-Qwen-32B ─────────────────────────────────────────────────────────╯

────────────────────────────────────────────────── Initial plan ───────────────────────────────────────────────────
Here is the plan of action that I will follow to solve the task:
```
Okay, so I need to figure out how to solve this task step by step. The goal is to find all Batman filming locations
worldwide, calculate the transfer time via cargo plane from Gotham, find supercar factories with the same transfer 
time, and plot them on a map. Let's break it down.

First, I need the list of Batman filming locations. I'm not sure where to find that, so I'll probably use a web 
agent to search for "Batman filming locations list" or check IMDb or Wikipedia. Once I have the list, I'll need the
exact locations, maybe names of cities or specific addresses.

Next, I need the coordinates for each location. If the web agent gives me addresses, I can use geocoding services 
like Google Maps API or an online tool to get latitude and longitude. Gotham's coordinates are given, so that's our
starting point.

Now, calculating the cargo plane transfer time. I remember the formula uses great-circle distance, which is the 
shortest distance on Earth. I should find an average cargo plane speed—maybe around 750 km/h? But I'm not sure, so 
I'll look it up or use a tool to calculate travel time using the distance.

Once I have all filming locations and their flight times from Gotham, I need to find supercar factories with the 
same transfer times. I don't have a list of supercar factories, so again, I'll use the web agent to find a list and
their locations. Then, I'll calculate the transfer time from Gotham to each factory and see which ones match the 
Batman locations' times. If I don't have enough points, I might need more locations from either category.

For the visualization, I'll use Plotly as in the example. I'll create a DataFrame with latitude, longitude, names, 
and transfer times. Then, I'll plot them with colors based on transfer time, using the Magma color scale. I need to
make sure there are at least six points in total. After plotting, I'll save the map as saved_map.png.

I should also make sure that each step is properly handled, especially data collection and accuracy of coordinates.
Maybe some locations are too vague, so I'll need precise data to get accurate distances and times. Also, I'll need 
to handle any errors, like if a location can't be found or if the web agent can't retrieve the data.

Finally, after all data is collected and processed, I'll compile everything into the map and ensure it meets the 
requirements before finalizing the answer.
</think>

**Step-by-step Plan:**

1. **Collect Batman Filming Locations:**
   - Use a web agent to find a comprehensive list of Batman filming locations worldwide, including specific names 
and locations.

2. **Obtain Coordinates:**
   - For each filming location, use geocoding services or online tools to determine the exact latitude and 
longitude.

3. **Calculate Transfer Times from Gotham:**
   - Using the provided function `calculate_cargo_travel_time`, compute the flight time for each location from 
Gotham (40.7128° N, 74.0060° W) using the great-circle distance formula and an average cargo plane speed.

4. **Identify Supercar Factories:**
   - Utilize the web agent to gather a list of supercar factories, including their names and locations.

5. **Determine Transfer Times for Factories:**
   - Calculate the transfer time from Gotham to each supercar factory using the same method as for filming 
locations.

6. **Match Transfer Times:**
   - Compare the transfer times of supercar factories with those of Batman locations to find at least six points 
that have the same travel time.

7. **Prepare Data for Visualization:**
   - Compile all selected locations into a DataFrame with columns for latitude, longitude, name, and transfer time.

8. **Generate the Map:**
   - Using Plotly Express, create a scatter map with points colored by transfer time, utilizing the Magma color 
scale. Adjust 

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

Output message of the LLM: ────────────────────────────────────────────────────────────────────────────────────────
Alright, let's tackle this problem step by step. The user wants to find Batman filming locations worldwide,        
calculate the travel time via cargo plane from Gotham, and also identify supercar factories with similar travel    
times. Finally, they want a spatial map of these locations.                                                        
                                                                                                                   
First, I need to gather Batman filming locations. I'll use a web agent since it can search and extract data from   
the web efficiently. I'll ask it to provide a list with names and locations, ensuring there are at least 6 points. 
                                                                                                                   
Next, for each of these locations, I need their coordinates. Web agents can help with geocoding, so I'll instruct  
it to find the latitude and longitude for each place. Once I have the coordinates, I can use the                   
`calculate_cargo_travel_time` tool to compute the travel time from Gotham, which is located at (40.7128°N,         
74.0060°W).                                                                                                        
                                                                                                                   
While I'm gathering Batman locations, I'll also search for supercar factories. I'll ask the web agent to list these
factories, including their locations. Once I have their names and locations, I'll again use geocoding to get their 
coordinates and then compute the travel times from Gotham.                                                         
                                                                                                                   
The next challenge[38;2;23

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  batman_locations_task = """                                                                                      
  Please research and provide a list of at least 6 Batman filming locations worldwide, including their names and   
  exact locations (cities or specific addresses).                                                                  
  """                                                                                                              
  web_agent(task=batman_locations_task)                                                                            
                                                                                                                   
  supercar_factories_task = """                                                                                    
  Please provide a list of at least 6 supercar factories worldwide, including their names and exact locations      
  (cities or specific addresses).                                                                                  
  """                                                                                                              
  web_agent(task=supercar_factories_task)                                                                          
                                                                                                                   
  geocode_task = """                                                                                               
  Please provide the latitude and longitude for the following locations:                                           
  """                                                                                                              
  web_agent(task=geocode_task)                                                                                     
                                                                                                                   
  from math import radians, sin, cos, sqrt, atan2                                                                  
  import pandas as pd                                                                                              
                                                                                                                   
  # List of coordinates                                                                                            
  locations = {                                                                                                    
      'Batman Locations': [                                                                                        
          (40.7128, -74.0060),  # Gotham (example)                                                                 
          (51.5074, -0.1278),   # Example location 1                                                               
          (48.8566, 2.3522),    # Example location 2                                                               
          # Add all Batman locations here                                                                          
      ],                                                                                                           
      'Supercar Factories': [                                                                                      
          (48.8566, 2.3522),    # Example location 1                                                               
          (51.5074, -0.1278),   # Example location 2                                                               
          # Add all supercar factories here                                                                        
      ]                                                                                                            
  }                                                      

I don't know how that went in your run, but in mine, the manager agent skilfully divided tasks given to the web agent in `1. Search for Batman filming locations`, then `2. Find supercar factories`, before aggregating the lists and plotting the map.

Let's see what the map looks like by inspecting it directly from the agent state:

In [23]:
manager_agent.python_executor.state["fig"]

KeyError: 'fig'

![output map](https://huggingface.co/datasets/agents-course/course-images/resolve/main/en/unit2/smolagents/output_map.png)